In [5]:
pip install trtokenizer

In [6]:
import pandas as pd
import re
import string
from trtokenizer.tr_tokenizer import SentenceTokenizer
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
from snowballstemmer import stemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import spacy
from spacy.lang.tr.stop_words import STOP_WORDS



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
df = pd.read_excel('tweetData.xlsx')

In [8]:
df = df.dropna()

In [9]:
df.shape

(1976, 2)

In [10]:
def remove_special_characters(text):
    try:
      text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
      return text
    except:
        pass

def to_lowercase(text):
    try:
      return text.lower()
    except:
        pass

def remove_extra_spaces(text):
    try:
      return  ' '.join(text.split())
    except:
        pass

def remove_html_tags(text):
    try:
      clean = re.compile('<.*?>')
      return re.sub(clean, '', text)
    except:
        pass

def remove_urls(text):
    try:
      return re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    except:
        pass

def remove_numbers(text):
    try:
      return re.sub(r'\d+', '', text)
    except:
        pass

def remove_punctuation(text):
    try:
      translator = str.maketrans('', '', string.punctuation)
      return text.translate(translator)
    except:
      pass

In [11]:
df['Tweets'] = df['Tweets'].apply(to_lowercase).apply(remove_extra_spaces).apply(remove_punctuation).apply(remove_html_tags).apply(remove_urls).apply(remove_numbers)

<ipython-input-11-acdcc727262d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Tweets'] = df['Tweets'].apply(to_lowercase).apply(remove_extra_spaces).apply(remove_punctuation).apply(remove_html_tags).apply(remove_urls).apply(remove_numbers)


In [12]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
df.Tweets.loc[15595:15599]

15595    kimmiş lan rahatsız olacak söylesin infaz edel...
15596                              i̇nstasina infaz edelim
15597    üstad latin abd ülkelerinde denediler direk ka...
15598    bu iki oevladının açık adreslerini verin infaz...
15599     kıbrısın tamamı alınmalı   ada diye bize yutt...
Name: Tweets, dtype: object

In [13]:
df.head()

,Tweets,Etiket
0,aynı bayrak aynı göğün altındayız biriz berabe...,1.0
1,çok istekli olan bu erkekler lütfen erkek çoc...,1.0
2,kurtlerle komşumuz alrabayiz beraberiz siz yal...,2.0
3,müslümanın seni beni ırkı olmaz dini olur kims...,1.0
4,soramazsına hoşgeldiniz bu gün bilge ile berab...,2.0


In [15]:


# Cümle ve kelime ayrıştırıcı işlevler
def trtokenizer_sentence(text):
    try:
        return sent_tokenize(text, language='turkish')
    except Exception as e:
        print(f"Sentence tokenization error: {e}")
        return []

def trtokenizer_word(text):
    try:
        return word_tokenize(text, language='turkish')
    except Exception as e:
        print(f"Word tokenization error: {e}")
        return []

def remove_stopwords_spacy(text):
    turkish_stopwords = STOP_WORDS
    cleaned = [word for word in text if word not in turkish_stopwords]
    return cleaned

def snowballstemmer(text):

  # Snowball Stemmer oluşturulması
  turkish_stemmer = stemmer("turkish")
  # Stem yapılması
  stemmed_words = [turkish_stemmer.stemWord(word) for word in text]
  return ' '.join(stemmed_words)




# Cümle ve kelime ayrıştırma işlemlerini DataFrame'e uygulama
df['sentences'] = df['Tweets'].apply(trtokenizer_sentence)

# Kelimeleri çıkarmak için her cümleyi kelimelere böler ve tüm kelimeleri düz bir liste olarak toplar
df['words'] = df['sentences'].apply(lambda sentences: [word for sentence in sentences for word in trtokenizer_word(sentence)])

# Stop kelimeleri temizleme işlemini ve birleştirme işlemini DataFrame'e uygulama
df['cleaned_words'] = df['words'].apply(remove_stopwords_spacy)
df['lemmatized_words'] = df['cleaned_words'].apply(snowballstemmer)

# Sonuçları yazdırma
print(df.head())




                                              Tweets  Etiket  \
0  aynı bayrak aynı göğün altındayız biriz berabe...     1.0   
1  çok istekli olan bu erkekler  lütfen erkek çoc...     1.0   
2  kurtlerle komşumuz alrabayiz beraberiz siz yal...     2.0   
3  müslümanın seni beni ırkı olmaz dini olur kims...     1.0   
4  soramazsına hoşgeldiniz bu gün bilge ile berab...     2.0   

                                           sentences  \
0  [aynı bayrak aynı göğün altındayız biriz berab...   
1  [çok istekli olan bu erkekler  lütfen erkek ço...   
2  [kurtlerle komşumuz alrabayiz beraberiz siz ya...   
3  [müslümanın seni beni ırkı olmaz dini olur kim...   
4  [soramazsına hoşgeldiniz bu gün bilge ile bera...   

                                               words  \
0  [aynı, bayrak, aynı, göğün, altındayız, biriz,...   
1  [çok, istekli, olan, bu, erkekler, lütfen, erk...   
2  [kurtlerle, komşumuz, alrabayiz, beraberiz, si...   
3  [müslümanın, seni, beni, ırkı, olmaz, dini, ol...  

In [16]:

# TfidfVectorizer ile metinleri vektörleştir
tfidf_vectorizer = TfidfVectorizer()
tfidf_vectors = tfidf_vectorizer.fit_transform(df['lemmatized_words'])
print(tfidf_vectors)


  (0, 267)	0.17367738762021206
  (0, 2886)	0.18050574709276337
  (0, 6101)	0.21454404700774177
  (0, 2885)	0.21454404700774177
  (0, 2718)	0.19571083417302798
  (0, 6476)	0.17081468100982655
  (0, 888)	0.21454404700774177
  (0, 7752)	0.21454404700774177
  (0, 1776)	0.21454404700774177
  (0, 3068)	0.14485287821396226
  (0, 874)	0.18050574709276337
  (0, 2280)	0.14702768522719242
  (0, 3568)	0.18964789384454037
  (0, 6388)	0.16586089806190624
  (0, 411)	0.1360109619507845
  (0, 84)	0.17367738762021206
  (0, 7461)	0.2035273237313338
  (0, 7587)	0.21454404700774177
  (0, 592)	0.19571083417302798
  (0, 4487)	0.25203933075849694
  (0, 4168)	0.15198146817511277
  (0, 330)	0.19571083417302798
  (0, 954)	0.1360109619507845
  (0, 1046)	0.14614722103522915
  (0, 247)	0.1506558109816416
  :	:
  (1974, 5805)	0.22192174768256562
  (1974, 6849)	0.21339881822169587
  (1974, 1582)	0.21339881822169587
  (1974, 3470)	0.3668578345689992
  (1974, 2069)	0.130798192505508
  (1974, 1116)	0.17847973790880883
 